In [20]:
import pandas as pd
import numpy as np
import wrangle
import warnings
warnings.filterwarnings("ignore")

import wrangle
from pydataset import data

1) Load the tips dataset.

In [91]:
tips = data('tips')

In [92]:
# split my tips data
tips_train, tips_validate, tips_test = wrangle.split_continuous(tips)

train -> (136, 7)
validate -> (59, 7)
test -> (49, 7)


1a) Create a column named tip_percentage. This should be the tip amount divided by the total bill.

In [93]:
tips_train['tip_percentage'] = tips_train['tip'] / tips_train['total_bill']

In [94]:
tips_train.head()

,total_bill,tip,sex,smoker,day,time,size,tip_percentage
19,16.97,3.50,Female,No,Sun,Dinner,3,0.206246
173,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345
119,12.43,1.80,Female,No,Thur,Lunch,2,0.144811
29,21.70,4.30,Male,No,Sat,Dinner,2,0.198157
238,32.83,1.17,Male,Yes,Sat,Dinner,2,0.035638


1b) Create a column named price_per_person. This should be the total bill divided by the party size.

In [95]:
tips_train['price_per_person'] = tips_train['total_bill'] / tips_train['size']

In [96]:
tips_train.head()

,total_bill,tip,sex,smoker,day,time,size,tip_percentage,price_per_person
19,16.97,3.50,Female,No,Sun,Dinner,3,0.206246,5.656667
173,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345,3.625000
119,12.43,1.80,Female,No,Thur,Lunch,2,0.144811,6.215000
29,21.70,4.30,Male,No,Sat,Dinner,2,0.198157,10.850000
238,32.83,1.17,Male,Yes,Sat,Dinner,2,0.035638,16.415000


In [97]:
# Split data sets into target and features
X_train = tips_train.drop(columns=['tip'])
y_train = tips_train['tip']
X_validate = tips_validate.drop(columns=['tip'])
y_validate = tips_validate['tip']
X_test = tips_test.drop(columns=['tip'])
y_test = tips_test['tip']

1c) Before using any of the methods discussed in the lesson, which features do you think would be most important for predicting the tip amount? The tip percentage?

In [98]:
# tip percentage and price per person

1d) Use all the other numeric features to predict tip amount. Use select k best and recursive feature elimination to select the top 2 features. What are they?

In [99]:
X_train_num = X_train.drop(columns=['sex', 'smoker', 'day', 'time'])

In [100]:
X_train_num.head()

,total_bill,size,tip_percentage,price_per_person
19,16.97,3,0.206246,5.656667
173,7.25,2,0.710345,3.625000
119,12.43,2,0.144811,6.215000
29,21.70,2,0.198157,10.850000
238,32.83,2,0.035638,16.415000


In [101]:
# selecting two most predictive features using SelectKBest

from sklearn.feature_selection import SelectKBest, f_regression

In [150]:
# parameters: f_regression stats test, give me 2 features
f_selector = SelectKBest(f_regression, k=3)

# find the top 8 X's correlated with y
f_selector.fit(X_train_num, y_train)

# boolean mask of whether the column was selected or not. 
feature_mask = f_selector.get_support()

# get list of top K features. 
f_feature = X_train_num.iloc[:,feature_mask].columns.tolist()

In [151]:
f_feature

['total_bill', 'size', 'tip_percentage']

In [146]:
# selecting two most predictive feature using Recursive Feature Elimination

from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

# initialize the ML algorithm
lm = LinearRegression()

# create the rfe object, indicating the ML object (lm) and the number of features I want to end up with. 
rfe = RFE(lm, 3)

# fit the data using RFE
rfe.fit(X_train_num, y_train)  

# get the mask of the columns selected
feature_mask = rfe.support_

# get list of the column names. 
rfe_feature = X_train_num.iloc[:,feature_mask].columns.tolist()

In [147]:
rfe_feature

['total_bill', 'size', 'tip_percentage']

1e) Use all the other numeric features to predict tip percentage. Use select k best and recursive feature elimination to select the top 2 features. What are they?

In [148]:
# Split data sets into target and features
X_train2 = tips_train.drop(columns=['tip_percentage'])
y_train2 = tips_train['tip_percentage']

In [167]:
X_train_num2 = X_train2.select_dtypes(exclude=['object'])
X_train_num2.head()

,total_bill,tip,size,price_per_person
19,16.97,3.50,3,5.656667
173,7.25,5.15,2,3.625000
119,12.43,1.80,2,6.215000
29,21.70,4.30,2,10.850000
238,32.83,1.17,2,16.415000


In [152]:
# selecting two most predictive features using SelectKBest

# parameters: f_regression stats test, give me 2 features
f_selector = SelectKBest(f_regression, k=3)

# find the top 8 X's correlated with y
f_selector.fit(X_train_num2, y_train2)

# boolean mask of whether the column was selected or not. 
feature_mask = f_selector.get_support()

# get list of top K features. 
f_feature = X_train_num2.iloc[:,feature_mask].columns.tolist()

In [153]:
f_feature

['total_bill', 'tip', 'price_per_person']

In [154]:
# selecting two most predictive feature using Recursive Feature Elimination

# initialize the ML algorithm
lm = LinearRegression()

# create the rfe object, indicating the ML object (lm) and the number of features I want to end up with. 
rfe = RFE(lm, 3)

# fit the data using RFE
rfe.fit(X_train_num2, y_train2)  

# get the mask of the columns selected
feature_mask = rfe.support_

# get list of the column names. 
rfe_feature = X_train_num2.iloc[:,feature_mask].columns.tolist()

In [155]:
rfe_feature

['total_bill', 'tip', 'size']

1f) Why do you think select k best and recursive feature elimination might give different answers for the top features? Does this change as you change the number of features your are selecting?

In [156]:
# The iterative nature of the RFE method allows eliminates noise/correlations between features allowing for a more accurate selection process.  Whereas, the KBest selects the top features in one single iteration.  The two methods will tend to converge on the same features as the number of features selected increases.


2) Write a function named select_kbest that takes in the predictors (X), the target (y), and the number of features to select (k) and returns the names of the top k selected features based on the SelectKBest class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [ ]:
def select_kbest(df, k):
    #DF split
    df_train, df_validate, df_test = wrangle.split_continuous(df)
    #select numeric columns from df
    X_train_num = df_train.select_dtypes(exclude=['object'])
    
    
    X_reduced2 = SelectKBest(f_regression, k=2).fit_transform(X_train_scaled, y_train)